In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

## Reading excel file


In [9]:
xls = pd.ExcelFile('data.xlsx')

## Reading individual Sheets

In [23]:
# Industrialisation DataFrame
ind = pd.read_excel(xls, 'industrialisation')
ind.head()
ind.shape

(2840, 30)

In [19]:
# CO2 dataframe
co2 = pd.read_excel(xls, 'Co2')
co2.head()

,Entity,Code,Year,Annual CO2 emissions (tonnes )
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0


In [21]:
co2.shape

(20853, 4)

In [25]:
# Global Temps Dataframe
gltemp = pd.read_excel(xls, 'GlobalLandTemperaturesByCity')
gltemp.head()
gltemp.shape

(1048575, 7)

In [ ]:
chck = ['90','91','92','93','94','95','96','97','98','99']
chck2 = ['00','01','02','03','04','05','06','07','08','09','10']
dates = list(gltemp.iloc[:,0])

for i in range(len(dates)):
  if( not ( (dates[i][-2:] in chck or dates[i][-2:] in chck2) and dates[i][0:2] == "01/" )):
    gltemp.drop([i], inplace = True, axis = 0) 
print(gltemp)